<a href="https://colab.research.google.com/github/Blackman9t/Coursera_Capstone/blob/master/applied_data_science_capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Applied Data Science Capstone-Project


### This Capsone-Project is the final course in a series of nine(9) Data Science Specialization courses taught by IBM via Coursera. <br>
###The courses include the following:-
1. [What is Data Science?](https://www.coursera.org/learn/what-is-datascience/home/welcome)
2. [Open Source tools for Data Science](https://www.coursera.org/learn/open-source-tools-for-data-science/home/welcome)
3. [Python for Data Science](https://www.coursera.org/learn/python-for-applied-data-science/home/welcome)
4. [Data Science Methodology](https://www.coursera.org/learn/data-science-methodology/home/welcome)
5. [Databases and SQL for Data Science](https://www.coursera.org/learn/sql-data-science/home/welcome)
6. [Data Visualization with Python](https://www.coursera.org/learn/python-for-data-visualization/home/welcome)
7. [Data Analysis with Python](https://www.coursera.org/learn/data-analysis-with-python/home/welcome)
8. [Machine Learning with Python](https://www.coursera.org/learn/machine-learning-with-python/home/welcome)
9. [Applied Data Science Capstone](https://www.coursera.org/learn/applied-data-science-capstone/home/welcome)

#### It is mandatory that all courses from 1 to 8 are passed with acceptable grades before attempting this Capsone-Project.

Importing requred libraries

In [0]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
